<h1>Capstone Project - Week 3</h1>
<em>Alex Lynn</em>

### Imports

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup

### Get Data From Website

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = urllib.request.urlopen(url).read()

In [3]:
soup = BeautifulSoup(data, 'lxml')

In [4]:
table = soup.find('table', class_='wikitable sortable')

In [5]:
postalcode = []
borough = []
neighbourhood = []

for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postalcode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighbourhood.append(cells[2].find(text=True))


In [6]:
df = pd.DataFrame(postalcode,columns=['PostalCode'])
df['Borough'] = borough
df['Neighbourhood'] = neighbourhood

df.shape

(180, 3)

### Data Processing

In the following cell I am removing whitespace and \\n from the data we have retrieved.<br>
Then finally I have dropped any Boroughs' which were Not Assigned.

In [7]:
# Trim whitespace
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Remove \n from columns
df.replace(r'\n',' ', regex=True, inplace=True) 

# Change all Not assigned in Neighbourhood to Borough
df['Neighbourhood'].replace('Not Assigned', df['Borough'], inplace=True)

# Remove Not Assigned from Borough
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape

(103, 3)

#### 77 Rows have been removed as they did not have a borough assigned.

## Add Longitude and Latitude to DataFrame

In [9]:
!pip3 install geocoder

Defaulting to user installation because normal site-packages is not writeable


### Imports

In [10]:
import geocoder

In [ ]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]